In [2]:
%load_ext autoreload
%autoreload 2

In [130]:
import argiope, compmod2
import pandas as pd
import numpy as np
from string import Template

In [189]:
def _unsorted_set(df, label, maxwidth = 40 ):
    """
    Returns a set as inp string with unsorted option.
    """
    out = "*NSET, NSET={0}, UNSORTED\n".format(label)
    labels = df.index.values
    line = "  "
    counter = 2
    for l in labels:
        counter += 1
        s = "{0},".format(l)
        if (len(s) + len(line) < maxwidth) and counter < maxwidth:
            line += s
        else:
            out += line + "\n  "
            line = s
            counter = 2
    out += line + "\n"
    return out.strip()[:-1]          

class CuboidSample(argiope.models.Part):
    """
    A cuboid test sample.
    """
    def __init__(self, shape = (1, 1, 1), dim = (1., 1., 1.)):
        self.dim = dim
        self.shape = shape

    def make_mesh(self):
        """
        Makes the mesh
        """    
        mesh = argiope.mesh.structured_mesh(dim = self.dim, shape = self.shape)
        self.mesh = mesh

In [190]:
'''
def get_node_pairs(pair, normal = "x"):
    """
    Returns pairs of nodes that are in the same position on two faces modulo
    a translation along a normal direction.
    """
    plane = ["x", "y", "z"]
    plane = [d for d in plane if d != normal]
    pair = [left, right]
    pair = [d.coords[plane].sort_values(plane) for d in pair]
    return np.array([d.index for d in pair]).T        

get_node_pairs(pair = [front, back] , normal = "z").shape'''
0

0

In [207]:
main_template = """
********************************************************************************
**DISTRIBUTED MECHANICAL PROPERTIES ********************************************
********************************************************************************
**HEADER
*Preprint, echo=NO, model=NO, history=NO, contact=NO
********************************************************************************
** PART "pSAMPLE" DEFINITION
*Part, name = pSample
$MESH
********************************************************************************
**BOUNDARY CONDITIONS **********************************************************
********************************************************************************
$BOUNDARY_CONDITIONS
*End part
********************************************************************************
** ASSEMBLY ********************************************************************
********************************************************************************
*Assembly, name = Assembly
*Instance, name=iSample, part=pSample
*End Instance
*End Assembly
********************************************************************************
** MATERIALS *******************************************************************
********************************************************************************
#MATERIALS
********************************************************************************
** STEPS ***********************************************************************
********************************************************************************
"""


In [218]:
sample = CuboidSample(shape = (3,3,3))
sample.make_mesh()
m = sample.mesh
n = m.nodes
c = n.coords
xm, ym, zm = c.max(axis = 0)
# CORNERS
n[("sets", "pinned")]    = (c.x == 0.) & (c.y == 0.) & (c.z == 0.)
n[("sets", "controled")] = (c.x == xm) & (c.y == ym) & (c.z == zm)
n[("sets", "zpinned")]   = (c.x == xm) & (c.y == 0.) & (c.z == 0.)

m.nodes.head()
m.elements.materials = ["mat{0}".format(i) for i in m.elements.index]
m.elements.loc[:, ("type", "solver", "") ] = "C3D8"

# BOUNDARY CONDITIONS
faces = {
        "left"   : {"nodes": n.coords[(c.x == 0.)].sort_values(["y", "z"]), "plane": [2,3], "normal":1},
        "right"  : {"nodes": n.coords[(c.x == xm)].sort_values(["y", "z"]), "plane": [2,3], "normal":1},
        "bottom" : {"nodes": n.coords[(c.y == 0.)].sort_values(["x", "z"]), "plane": [1,3], "normal":2},
        "top"    : {"nodes": n.coords[(c.y == ym)].sort_values(["x", "z"]), "plane": [1,3], "normal":2}, 
        "front"  : {"nodes": n.coords[(c.z == 0.)].sort_values(["x", "y"]), "plane": [1,2], "normal":3},  
        "back"   : {"nodes": n.coords[(c.z == zm)].sort_values(["x", "y"]), "plane": [1,2], "normal":3},  
        }
 

equation_pattern = "*EQUATION\n  {0}\n"

bc = "\n".join([_unsorted_set(v["nodes"], k) for k, v in faces.items()]) + "\n"
pairs = [["left",   "right"],
         ["bottom", "top"  ],
         ["back",   "front"],]
for pair in pairs:
    f = [faces[pair[0]], faces[pair[1]]] 
    for axis in f[0]["plane"]:
        bc += equation_pattern.format(2)
        bc += "  {0}, {1}, 1., {2}, {3}, -1.\n".format(pair[0], axis, pair[1], axis)

bc_type = "periodic"
if bc_type == "periodic":
    for pair in pairs:
        f = [faces[pair[0]], faces[pair[1]]] 
        axis = f[0]["normal"]
        bc += equation_pattern.format(2)
        bc += "  {0}, {1}, 1., {2}, {3}, -1.\n".format(pair[0], axis, pair[1], axis)

out = Template(main_template).substitute(
    MESH = sample.mesh.write_inp(),
    BOUNDARY_CONDITIONS = bc.strip("\n"),
    )

    
print(out)


********************************************************************************
**DISTRIBUTED MECHANICAL PROPERTIES ********************************************
********************************************************************************
**HEADER
*Preprint, echo=NO, model=NO, history=NO, contact=NO
********************************************************************************
** PART "pSAMPLE" DEFINITION
*Part, name = pSample
********************************************************************************
** NODES ***********************************************************************
********************************************************************************
*NODE
  1, 0.0, 0.0, 0.0
  2, 0.3333333333333333, 0.0, 0.0
  3, 0.6666666666666666, 0.0, 0.0
  4, 1.0, 0.0, 0.0
  5, 0.0, 0.3333333333333333, 0.0
  6, 0.3333333333333333, 0.3333333333333333, 0.0
  7, 0.6666666666666666, 0.3333333333333333, 0.0
  8, 1.0, 0.3333333333333333, 0.0
  9, 0.0, 0.6666666666666666, 0.0
  10, 0

In [196]:
n.coords

,x,y,z
node,,,
1,0.000000,0.000000,0.000000
2,0.333333,0.000000,0.000000
3,0.666667,0.000000,0.000000
4,1.000000,0.000000,0.000000
5,0.000000,0.333333,0.000000
6,0.333333,0.333333,0.000000
7,0.666667,0.333333,0.000000
8,1.000000,0.333333,0.000000
9,0.000000,0.666667,0.000000



**----------------------------------
**DISTRIBUTED MECHANICAL PROPERTIES
**----------------------------------
**HEADER
*Preprint, echo=NO, model=NO, history=NO, contact=NO
**----------------------------------
** PART "pSAMPLE" DEFINITION
*Part, name = pSample
********************************************************************************
** NODES ***********************************************************************
********************************************************************************
*NODE
  1, 0.0, 0.0, 0.0
  2, 0.5, 0.0, 0.0
  3, 1.0, 0.0, 0.0
  4, 0.0, 0.5, 0.0
  5, 0.5, 0.5, 0.0
  6, 1.0, 0.5, 0.0
  7, 0.0, 1.0, 0.0
  8, 0.5, 1.0, 0.0
  9, 1.0, 1.0, 0.0
  10, 0.0, 0.0, 0.5
  11, 0.5, 0.0, 0.5
  12, 1.0, 0.0, 0.5
  13, 0.0, 0.5, 0.5
  14, 0.5, 0.5, 0.5
  15, 1.0, 0.5, 0.5
  16, 0.0, 1.0, 0.5
  17, 0.5, 1.0, 0.5
  18, 1.0, 1.0, 0.5
  19, 0.0, 0.0, 1.0
  20, 0.5, 0.0, 1.0
  21, 1.0, 0.0, 1.0
  22, 0.0, 0.5, 1.0
  23, 0.5, 0.5, 1.0
  24, 1.0, 0.5, 1.0
  25, 0.0, 1.0, 1.0

In [152]:
m.elements


conn                             materials  sets    type       
          n0  n1  n2  n3  n4  n5  n6  n7             all argiope solver
                                                                       
element                                                                
1          1   2   5   4  10  11  14  13      mat1  True   hexa8       
2          2   3   6   5  11  12  15  14      mat2  True   hexa8       
3          4   5   8   7  13  14  17  16      mat3  True   hexa8       
4          5   6   9   8  14  15  18  17      mat4  True   hexa8       
5         10  11  14  13  19  20  23  22      mat5  True   hexa8       
6         11  12  15  14  20  21  24  23      mat6  True   hexa8       
7         13  14  17  16  22  23  26  25      mat7  True   hexa8       
8         14  15  18  17  23  24  27  26      mat8  True   hexa8

In [177]:
30. * 41

1230.0

In [178]:
20 * 41

820